In [19]:
import os
import json
import fitz #(PyMuPDF)
from tqdm import tqdm

La méthode présentée permet de sauvegarder plus facilement les résultats au format json, en séparant les informations de chaque page du document pdf. Étant donné que l'invite prévue pour le traitement ultérieur du résultat implique de travailler sur un fichier csv, nous allons d'abord extraire le document json, puis le fichier csv.

!! Dans les situations quand pendant le traitement de fichier fitz(PyMuPDF) retourne un erreur, le nom de ce fichier est sauvegardé dans un fichier txt. Ce sont probablement les fichier dont le texte est impossible pour l'extraction par ce methode - il faudra les traiter par le methode 2. 

In [ ]:
base_dir = os.path.expanduser("~/Documents/hackaton_week/ScincePo_data")
out_jsonl = os.path.join(base_dir, "pdf_pages_extracted_checked.jsonl")

#On indique les dossiers qu'on veut ignorer : dans ce cas ce sont les dossiers avec les fichiers _falc et les fichiers PDF-images sans text déjà extraits
skip_names = {"_falc", "broken_pdf", "no_text", "broken_last"}

In [73]:
# Désactiver l’affichage des avertissements et erreurs MuPDF
fitz.TOOLS.mupdf_display_errors(False)
fitz.TOOLS.mupdf_display_warnings(False)

# Valeur utilisée pour remplacer les cellules vides
EMPTY_CELL_VALUE = "n"

# Fichier texte pour enregistrer les PDF provoquant des erreurs
error_txt = os.path.join(base_dir, "pdf_errors.txt")

#Constitution de la liste des fichiers PDF
pdf_paths = []
for root, dirs, files in os.walk(base_dir):
    # Exclure les dossiers non pertinents du parcours
    dirs[:] = [d for d in dirs if d not in skip_names]
    for f in files:
        if f.lower().endswith(".pdf"):
            pdf_paths.append(os.path.join(root, f))

pdf_paths = sorted(pdf_paths)

def normalize_cell(value: str | None) -> str:
    """
    Normalise les valeurs textuelles.
    Garantit qu’aucune chaîne vide n’est retournée.
    """
    v = (value or "").strip()
    return v if v else EMPTY_CELL_VALUE

#Identification des fichiers déjà traités
processed_filenames = set()
if os.path.exists(out_jsonl):
    with open(out_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                fn = obj.get("filename")
                if fn:
                    processed_filenames.add(fn)
            except json.JSONDecodeError:
                # Ignorer les lignes corrompues du fichier JSONL
                continue

print(f"Nombre de fichiers PDF déjà traités : {len(processed_filenames)}")

#Chargement des erreurs déjà enregistrées
logged_errors = set()
if os.path.exists(error_txt):
    with open(error_txt, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                logged_errors.add(line)

def log_error_filename(filename: str):
    """
    Enregistre le nom d’un fichier PDF ayant provoqué une erreur
    dans un fichier texte dédié, sans duplication.
    """
    if filename in logged_errors:
        return
    with open(error_txt, "a", encoding="utf-8") as f:
        f.write(filename + "\n")
    logged_errors.add(filename)

#Extraction du texte à partir des PDF (PyMuPDF)
def extract_pages(pdf_path: str, filename: str) -> list[str]:
    """
    Extrait le texte d’un fichier PDF page par page à partir des blocs de texte.
    En cas d’erreur d’ouverture ou de lecture, le fichier est consigné
    dans un fichier texte d’erreurs.
    """
    pages_out = []
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            blocks = page.get_text("blocks")
            clean = []

            for x0, y0, x1, y1, text, *_ in blocks:
                t = (text or "").strip()
                if not t:
                    continue
                clean.append((y0, x0, t))

            # Réorganisation du texte selon la position spatiale
            clean.sort(key=lambda k: (k[0], k[1]))
            page_text = "\n\n".join(t for _, _, t in clean)
            pages_out.append(normalize_cell(page_text))

        return pages_out

    except Exception:
        # Enregistrer le fichier PDF ayant causé une erreur
        log_error_filename(filename)
        return []

#Sauvegarde incrémentale au format JSONL
def flush_batch(rows: list[dict], jsonl_path: str):
    """
    Écrit un lot de résultats dans le fichier JSONL
    afin de limiter la perte de données en cas d’interruption.
    """
    if not rows:
        return
    with open(jsonl_path, "a", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

batch_rows = []
total_done = 0

#Boucle principale d’extraction (reprise possible)
for path in tqdm(pdf_paths, desc="Extraction du contenu des fichiers PDF", unit="pdf"):
    filename = os.path.basename(path)

    # Ignorer les fichiers déjà traités
    if filename in processed_filenames:
        continue

    pages = extract_pages(path, filename)

    # Garantir au moins une valeur non vide
    if not pages:
        pages = [EMPTY_CELL_VALUE]

    record = {
        "filename": normalize_cell(filename),
        "pages": [normalize_cell(p) for p in pages],
        "num_pages": len(pages),
    }

    batch_rows.append(record)
    processed_filenames.add(filename)
    total_done += 1

    # Sauvegarde périodique
    if total_done % 100 == 0:
        flush_batch(batch_rows, out_jsonl)
        batch_rows = []

# Écriture des résultats restants
flush_batch(batch_rows, out_jsonl)

print("Traitement terminé.")
print(f"Nombre de nouveaux fichiers PDF traités : {total_done}")
print(f"Fichier JSONL mis à jour : {out_jsonl}")
print(f"Liste des fichiers PDF en erreur : {error_txt}")


Déjà traités : 0 PDFs


Extraction des PDF: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11544/11544 [07:20<00:00, 26.23pdf/s]

Terminé. Nouveaux PDFs traités : 11544
JSONL mis à jour : /Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_checked.jsonl


Exemple de résultat JSON - on voit ici que le code met l'information sur deux pages dans une seule cellule, en les divisant par ".," - on va utiliser cette information pour séparer les pages et transformer le fichier dans le format csv. 

In [79]:
import json

obj = pd.read_json("/Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_checked.jsonl", lines = True)
obj

filename  \
0                       LG17-1-1-BONNOT-10-tour1-profession_foi.pdf   
1                        LG17-1-1-BRETON-5-tour1-profession_foi.pdf   
2                        LG17-1-1-BRETON-5-tour2-profession_foi.pdf   
3                       LG17-1-1-CARLIER-6-tour1-profession_foi.pdf   
4                      LG17-1-1-LÉPAGNOT-3-tour1-profession_foi.pdf   
...                                                             ...   
11539   LG24-ZZ-ZZ10-tour1-5-Marie Jos├йe MABASI_profession_foi.pdf   
11540    LG24-ZZ-ZZ10-tour1-8-Ali Camille HOJEIJ_profession_foi.pdf   
11541       LG24-ZZ-ZZ10-tour2-3-Am├йlia LAKRAFI_profession_foi.pdf   
11542           LG24-ZZ-ZZ10-tour2-6-Elsa DI MEO_profession_foi.pdf   
11543  LG24-ZZ-ZZ11-tour1-15-Tatiana BOTEVA MALO_profession_foi.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [80]:
in_jsonl = "/Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_checked.jsonl"
out_csv  = "/Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_split_pages.csv"

EMPTY = "n"
SEP = ".,"

df = pd.read_json(in_jsonl, lines=True)

def normalize_text(x: str) -> str:
    x = (x or "").strip()
    if not x:
        return EMPTY
    # чтобы CSV визуально не "ломался" от реальных переносов строк
    x = x.replace("\r\n", "\n").replace("\r", "\n").replace("\n", "\\n")
    return x

def split_pages_field(pages):
    """
    Возвращает list[str] страниц.
    - если pages уже list -> нормализуем элементы
    - если pages строка -> режем по SEP=".,"
    """
    if isinstance(pages, list):
        parts = [normalize_text(p) for p in pages]
        return parts if parts else [EMPTY]

    if pages is None:
        return [EMPTY]

    s = str(pages)
    # режем по точному маркеру между страницами
    raw_parts = [p.strip() for p in s.split(SEP)]
    raw_parts = [p for p in raw_parts if p]  # убрать пустые

    if not raw_parts:
        return [EMPTY]

    return [normalize_text(p) for p in raw_parts]

# 1) split
df["pages_list"] = df["pages"].apply(split_pages_field)

# 2) wide columns page_1..page_N
pages_wide = pd.DataFrame(df["pages_list"].tolist()).fillna(EMPTY)
pages_wide.columns = [f"page_{i+1}" for i in range(pages_wide.shape[1])]

# 3) финальный df
out_df = pd.concat([df[["filename"]].copy(), pages_wide], axis=1)
out_df["filename"] = out_df["filename"].astype(str).apply(normalize_text)
out_df = out_df.fillna(EMPTY)

# 4) сохранить "аккуратно"
out_df.to_csv(
    out_csv,
    index=False,
    encoding="utf-8",
    quoting=csv.QUOTE_ALL,
    escapechar="\\"
)

print("Saved:", out_csv)
print("Max pages:", pages_wide.shape[1])
print("Columns:", list(out_df.columns[:5]), "...")


Saved: /Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_split_pages.csv
Max pages: 7
Columns: ['filename', 'page_1', 'page_2', 'page_3', 'page_4'] ...


Exemple de résultat CSV

In [86]:
path = "/Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_split_pages.csv"
df = pd.read_csv(path)
df

filename  \
0                       LG17-1-1-BONNOT-10-tour1-profession_foi.pdf   
1                        LG17-1-1-BRETON-5-tour1-profession_foi.pdf   
2                        LG17-1-1-BRETON-5-tour2-profession_foi.pdf   
3                       LG17-1-1-CARLIER-6-tour1-profession_foi.pdf   
4                      LG17-1-1-LÉPAGNOT-3-tour1-profession_foi.pdf   
...                                                             ...   
11539   LG24-ZZ-ZZ10-tour1-5-Marie Jos├йe MABASI_profession_foi.pdf   
11540    LG24-ZZ-ZZ10-tour1-8-Ali Camille HOJEIJ_profession_foi.pdf   
11541       LG24-ZZ-ZZ10-tour2-3-Am├йlia LAKRAFI_profession_foi.pdf   
11542           LG24-ZZ-ZZ10-tour2-6-Elsa DI MEO_profession_foi.pdf   
11543  LG24-ZZ-ZZ11-tour1-15-Tatiana BOTEVA MALO_profession_foi.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Ici on supprime les pages >page_2 pour enlever l'information innécessaire. 

In [87]:
path = "/Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_split_pages.csv"
out_path = "/Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_page12_only.csv"

df = pd.read_csv(path)

# оставить только filename, page_1, page_2 (если page_2 нет — создаём)
cols_to_keep = ["filename", "page_1"]
if "page_2" in df.columns:
    cols_to_keep.append("page_2")
else:
    df["page_2"] = "n"
    cols_to_keep.append("page_2")

df = df[cols_to_keep]

# сохранить
df.to_csv(out_path, index=False, encoding="utf-8")

print("Saved:", out_path)
print("Columns:", df.columns.tolist())
print("Rows:", len(df))


Saved: /Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_page12_only.csv
Columns: ['filename', 'page_1', 'page_2']
Rows: 11544


Nettoyage du texte inutile mis automatiquement par modèle PyMuPDF

In [88]:
# Путь к файлу (поставь свой)
in_path = "/Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_page12_only.csv"
out_path = "/Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_page12_only_clean.csv"

df = pd.read_csv(in_path)

# Колонки "от 2-й" (всё кроме первой)
text_cols = df.columns[1:]

# Регекс: удаляем фразу TCPDF, допускаем разные пробелы/регистр
pattern = re.compile(r"Powered\s+by\s+TCPDF\s*\(www\.tcpdf\.org\)", flags=re.IGNORECASE)

# Чистим только текстовые колонки
for col in text_cols:
    # приводим к строке, чтобы .str работал даже если есть NaN/числа
    df[col] = df[col].astype("string").str.replace(pattern, "", regex=True).str.strip()

df.to_csv(out_path, index=False)
print("Sauvegardé:", out_path)
print("Lignes:", len(df))


Sauvegardé: /Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_page12_only_clean.csv
Lignes: 11544


Au cas où meme si après le traitement les fichiers qui renvoient l'erreur après le traitement par PyMuPDF il reste des fichiers mal traités par ce méthode, ce code permet de chercher ces fichers pour les traiter egalement par le methode 2. 

Critères de recherche : 
- Une des pages est vide
- Texte suspect de type “lettres aléatoires / capslock”
- Absence totale de caractères alphanumériques
- Seulement un seul mot détecté

In [55]:
import pandas as pd
import re

#chargement
df = pd.read_csv( "/Users/quentinnippert/Documents/hackaton_week/ScincePo_data/pdf_pages_extracted_FIXED_clean.csv")


In [59]:
"""
Ce script effectue un contrôle de qualité des données textuelles contenues dans
un DataFrame pandas. Il vise à identifier et isoler les lignes dont au moins une
cellule textuelle est vide, non informative ou manifestement bruitée.

L’analyse porte sur l’ensemble des colonnes textuelles (à l’exception de la
première colonne, supposée contenir un identifiant ou des métadonnées).
Une ligne est considérée comme invalide dès lors qu’au moins une de ses cellules
ne satisfait pas les critères minimaux de présence et de qualité du texte.

Les lignes invalides sont exportées dans un fichier CSV distinct, tandis que les
lignes valides sont conservées dans un second fichier.
"""

# Sélection des colonnes textuelles

# On considère toutes les colonnes sauf la première comme colonnes textuelles
text_columns = df.columns[1:]

# Expressions régulières

# Mot = au moins 2 lettres (alphabet latin avec diacritiques + cyrillique)
word_re = re.compile(r"[A-Za-zÀ-ÖØ-öø-ÿА-Яа-я]{2,}")

# Caractère alphanumérique (lettres ou chiffres)
alnum_re = re.compile(r"[0-9A-Za-zÀ-ÖØ-öø-ÿА-Яа-я]")

# Fonctions de validation

def est_colonne_invalide(val) -> bool:
    """
    Détermine si une cellule textuelle est invalide.

    Une cellule est considérée comme invalide si :
    - la valeur est manquante (NaN),
    - la chaîne est vide après nettoyage,
    - elle ne contient aucun caractère alphanumérique,
    - elle ne contient qu’un seul mot,
    - elle correspond à une suite suspecte de lettres (majuscules, bruit).
    """
    # Valeur manquante
    if pd.isna(val):
        return True

    s = str(val).strip()

    # 1) Chaîne vide
    if s == "":
        return True

    # 2) Absence totale de lettres ou de chiffres
    if not alnum_re.search(s):
        return True

    # Extraction des mots
    mots = word_re.findall(s)

    # 3) Un seul mot détecté
    if len(mots) == 1:
        return True

    # 4) Heuristique : suites de lettres aléatoires / majuscules
    letters_only = re.sub(r"[^A-Za-zÀ-ÖØ-öø-ÿА-Яа-я]", "", s)
    if letters_only:
        upper_ratio = sum(ch.isupper() for ch in letters_only) / len(letters_only)
        if len(mots) < 2 and len(letters_only) <= 30 and upper_ratio >= 0.8:
            return True

    return False


def ligne_a_sortir(row) -> bool:
    """
    Détermine si une ligne doit être isolée.

    Une ligne est marquée comme invalide si au moins une des
    colonnes textuelles contient une cellule invalide.
    """
    for col in text_columns:
        if est_colonne_invalide(row[col]):
            return True
    return False

# Application des critères

# Création du masque des lignes à isoler
masque_sortir = df.apply(ligne_a_sortir, axis=1)

# Séparation du DataFrame
df_a_sortir = df[masque_sortir]
df_reste = df[~masque_sortir]

# Informations de synthèse

print("Nombre total de lignes :", len(df))
print("Nombre de lignes isolées (contenu invalide) :", len(df_a_sortir))
print("Nombre de lignes conservées :", len(df_reste))

# Export des résultats

df_a_sortir.to_csv("lignes_a_sortir.csv", index=False)
df_reste.to_csv("lignes_restantes.csv", index=False)

print("\nFichiers générés :")
print("- lignes_a_sortir.csv (lignes rejetées)")
print("- lignes_restantes.csv (lignes conservées)")


Nombre total de lignes : 11671
Lignes sorties (critères 1/2/3) : 292
Lignes restantes : 11379

Fichiers enregistrés :
- lignes_a_sortir.csv
- lignes_restantes.csv
